In [1]:
import os
import csv
import numpy
import pandas
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import skfuzzy

In [2]:

data = {
    'current': [],
    'rawIntensity': [],
    'temperature': [],
    'voltage': [],
    'pwm': []
}

with open('dataset.csv', mode='r') as file:
    reader = csv.DictReader(file)

    for row in reader:
        data['current'].append(float(row['current']))
        data['rawIntensity'].append(int(row['rawIntensity']))
        data['temperature'].append(int(float(row['temperature'])))
        data['voltage'].append(float(row['voltage']))
        data['pwm'].append(int(row['pwm']))

df = pandas.DataFrame(data)

In [3]:
df.head()

,current,rawIntensity,temperature,voltage,pwm
0,37.5,1011,25,0.860,150
1,35.8,1004,25,0.900,100
2,37.7,1016,25,0.892,100
3,36.8,1010,25,0.892,100
4,37.1,1010,25,0.896,100


In [4]:
X = df[['current', 'rawIntensity', 'temperature', 'voltage']].values
y = df['pwm'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
def gaussian_mf(x, mean, sigma):
    return numpy.exp(-((x - mean) ** 2) / (2 * sigma ** 2))

def anfis_inference(inputs, mf_params, rule_weights):
    mf_values = []
    for i, input_value in enumerate(inputs):
        param = list(mf_params.values())[i]
        mf_values.append(gaussian_mf(input_value, param[0], param[1]))

    weighted_sum = 0
    weight_total = 0
    for i, weight in enumerate(rule_weights):
        weighted_sum += mf_values[i] * weight
        weight_total += weight

    return weighted_sum / weight_total if weight_total != 0 else 0



In [16]:
# Gaussian Membership Function
def gaussian_mf(x, mean, sigma):
    return numpy.exp(-((x - mean) ** 2) / (2 * sigma ** 2))

# Inferensi ANFIS
def anfis_inference(inputs, mf_params, rules):
    rule_outputs = []
    for rule in rules:
        # Hitung derajat keanggotaan untuk setiap input di aturan ini
        memberships = []
        for i, param in enumerate(rule['mf_params']):
            memberships.append(gaussian_mf(inputs[i], param[0], param[1]))

        # Kombinasikan menggunakan AND (T-norm)
        rule_strength = min(memberships)  # T-norm: Minimum

        # Hitung output aturan
        rule_outputs.append(rule_strength * rule['output'])

    # Defuzzifikasi menggunakan rata-rata berbobot
    weighted_sum = sum(rule_outputs)
    weight_total = sum([min(m) for m in rule_outputs])

    return weighted_sum / weight_total if weight_total != 0 else 0

In [7]:
# [mean, sigma]
mf_params = {
    'current': [0.6, 0.1],  
    'rawIntensity': [100, 10],
    'temperature': [30, 5],
    'voltage': [1.5, 0.2]
}

In [11]:
mf_params = [
    # Format: [mean, sigma]
    [[0.6, 0.1], [100, 10], [30, 5], [1.5, 0.2]],
    [[0.5, 0.2], [90, 20], [25, 6], [1.3, 0.3]]
]

rules = [
    {'mf_params': mf_params[0], 'output': 150},
    {'mf_params': mf_params[1], 'output': 100}
]

In [12]:
predictions = []
for x in X_test:
    predictions.append(anfis_inference(x, mf_params, rules))

# Evaluasi
mse = numpy.mean((numpy.array(predictions) - y_test) ** 2)
print(f"Mean Squared Error: {mse}")

TypeError: 'numpy.float64' object is not iterable

In [9]:
rule_weights = [1.0, 0.6, 0.6, 0.6]

predictions = []
for x in X_test:
    predictions.append(anfis_inference(x, mf_params, rule_weights))

mse = numpy.mean((numpy.array(predictions) - y_test) ** 2)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 13505.1845904679


In [20]:
# Gaussian Membership Function
def gaussian_mf(x, mean, sigma):
    return numpy.exp(-((x - mean) ** 2) / (2 * sigma ** 2))

# Inferensi ANFIS
def anfis_inference(inputs, mf_params, rules):
    rule_outputs = []
    for rule in rules:
        # Hitung derajat keanggotaan untuk setiap input di aturan ini
        memberships = []
        for i, param in enumerate(rule['mf_params']):
            memberships.append(gaussian_mf(inputs[i], param[0], param[1]))

        # Kombinasikan menggunakan AND (T-norm)
        rule_strength = min(memberships)  # T-norm: Minimum

        # Hitung output aturan
        rule_outputs.append(rule_strength * rule['output'])

    # Defuzzifikasi menggunakan rata-rata berbobot
    weighted_sum = sum(rule_outputs)
    weight_total = sum([m for m in rule_outputs])

    return weighted_sum / weight_total if weight_total != 0 else 0

In [21]:
# Dataset
data = {
    'current': [],
    'rawIntensity': [],
    'temperature': [],
    'voltage': [],
    'pwm': []
}

with open('dataset.csv', mode='r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        data['current'].append(float(row['current']))
        data['rawIntensity'].append(int(row['rawIntensity']))
        data['temperature'].append(int(float(row['temperature'])))
        data['voltage'].append(float(row['voltage']))
        data['pwm'].append(int(row['pwm']))

df = pandas.DataFrame(data)
X = df[['current', 'rawIntensity', 'temperature', 'voltage']].values
y = df['pwm'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Parameter Membership Functions dan Aturan
mf_params = [
    # Format: [mean, sigma]
    [[0.6, 0.1], [100, 10], [30, 5], [1.5, 0.2]],
    [[0.5, 0.2], [90, 20], [25, 6], [1.3, 0.3]]
]

rules = [
    {'mf_params': mf_params[0], 'output': 150},
    {'mf_params': mf_params[1], 'output': 100}
]

# Prediksi
predictions = []
for x in X_test:
    predictions.append(anfis_inference(x, mf_params, rules))

# Evaluasi
mse = numpy.mean((numpy.array(predictions) - y_test) ** 2)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 13533.849478390463


In [22]:
# Ekspor model ke file C header
with open("model.h", "w") as file:
    file.write("// Generated ANFIS Model\n\n")
    file.write("#ifndef ANFIS_MODEL_H\n#define ANFIS_MODEL_H\n\n")

    # Tulis parameter fungsi keanggotaan
    file.write("float mf_params[][2] = {\n")
    for params in mf_params:
        file.write("    {" + ", ".join(map(str, params[0])) + "},\n")
    file.write("};\n\n")

    # Tulis aturan fuzzy
    file.write("struct Rule {\n    float mf_params[4][2];\n    int output;\n};\n\n")
    file.write("Rule rules[] = {\n")
    for rule in rules:
        mf_str = ", ".join(
            "{" + ", ".join(map(str, param)) + "}" for param in rule['mf_params']
        )
        file.write(f"    {{{mf_str}, {rule['output']}}},\n")
    file.write("};\n\n")

    file.write("#endif // ANFIS_MODEL_H\n")


In [5]:
import csv
import pandas
import numpy
from sklearn.model_selection import train_test_split
import skfuzzy as fuzz

# Step 1: Load dataset
data = {
    'current': [],
    'rawIntensity': [],
    'temperature': [],
    'voltage': [],
    'pwm': []
}

with open('dataset.csv', mode='r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        data['current'].append(float(row['current']))
        data['rawIntensity'].append(int(row['rawIntensity']))
        data['temperature'].append(int(float(row['temperature'])))
        data['voltage'].append(float(row['voltage']))
        data['pwm'].append(int(row['pwm']))

df = pandas.DataFrame(data)

# Step 2: Split data into training and testing sets
X = df[['current', 'rawIntensity', 'temperature', 'voltage']].values
y = df['pwm'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Define fuzzy membership functions
def gaussian_mf(x, mean, sigma):
    return numpy.exp(-((x - mean) ** 2) / (2 * sigma ** 2))

# Initialize membership function parameters
mf_params = {
    'current': [0.6, 0.1],  # Example: [mean, sigma]
    'rawIntensity': [100, 10],
    'temperature': [30, 5],
    'voltage': [1.5, 0.2]
}

# Step 4: Train ANFIS model (rule-based, simple aggregation)
def anfis_inference(inputs, mf_params, rule_weights):
    mf_values = []
    for i, input_value in enumerate(inputs):
        param = list(mf_params.values())[i]
        mf_values.append(gaussian_mf(input_value, param[0], param[1]))
    
    # Combine membership function outputs using rules
    weighted_sum = 0
    weight_total = 0
    for i, weight in enumerate(rule_weights):
        weighted_sum += mf_values[i] * weight
        weight_total += weight

    return weighted_sum / weight_total if weight_total != 0 else 0

# Assign rule weights (example, could be learned in advanced ANFIS)
rule_weights = [1.0, 1.0, 1.0, 1.0]

# Step 5: Test model
predictions = []
for x in X_test:
    predictions.append(anfis_inference(x, mf_params, rule_weights))

# Calculate Mean Squared Error (MSE)
mse = numpy.mean((numpy.array(predictions) - y_test) ** 2)
print(f"Mean Squared Error: {mse}")

# Step 6: Export model to model.h
with open("model2.h", "w") as file:
    file.write("#ifndef MODEL_H\n#define MODEL_H\n\n")
    file.write("// Membership function parameters (mean, sigma)\n")
    file.write("float mf_params[4][2] = {\n")
    for key, params in mf_params.items():
        file.write(f"    {{{params[0]}, {params[1]}}}, // {key}\n")
    file.write("};\n\n")
    file.write("// Rule weights\n")
    file.write("float rule_weights[4] = {1.0, 1.0, 1.0, 1.0};\n\n")
    file.write("#endif // MODEL_H\n")


Mean Squared Error: 13499.787449995414
